# Project 2: Uncles and Uncles Rewards
- Model a Blockchain that uses uncles and uncle rewards
- Model rewarding mechanism to reward uncle block creators
- Model selfish mining (only one attacker) in this blockchain

Try to answer the following questions with your experiment:
1. How do uncles improve the fairness of the blockchain? For this, you should compare the outcome of miners with and without uncles.
2. What is the impact of uncles on selfish mining? Is selfish mining more profitable with uncles?
3. What does it mean in this model for the selfish mining attack to be profitable?

 Since more miners will get rewards for mining, even if they don't finish a block that is part of the longest chain, they will be rewarded for it. So the time and energy used for that block will not be as wasted, and it seems more rewarding to mine. Show net money with and without uncle rewards to display point. The concept of it is for fairness, but may encourage selfish mining. 

### Imports

In [52]:
#Import statements
from draw_blockchain import show
import hashlib as hasher
import random

### Hash function

In [53]:
def hashbits(input):
    hash_obj = hasher.sha256()
    inputbytes = input.encode()
    hash_obj.update(inputbytes)
    hashbytes = hash_obj.digest()
    return ''.join(f'{x:08b}' for x in hashbytes)

def hash(input):
    hash_obj = hasher.sha256()
    inputbytes = input.encode()
    #print(type(inputbytes))
    hash_obj.update(inputbytes)
    return hash_obj.hexdigest()

def numberOfInitZeros(hashStr):
    count = 0
    for i in range (0 , len(hashStr)):
        if hashStr[i] == '1':
            break
        count += 1
    return count

### Block

In [54]:
class Block:
    def __init__(self, data, creator=None, previous=None, nonce=0):
        self.data = data
        if previous is None:
            self.previous = None
            self.previous_hash = ""
            self.creator = Miner(0 , "0")
            self.height = 0
            self.color = "#AEF751"
        else:
            self.previous = previous
            self.previous_hash = previous.hash
            self.creator = creator
            self.height = previous.height+1
            self.color = creator.color
        self.nonce = nonce
        self.hash = self.hash_block()
        self.children = []

    def hash_block(self):
        return hashbits(self.data+ self.creator.name + self.previous_hash + str(self.nonce))

    def print(self):
        print(self.data + " "+ self.creator.name + " " + str(self.height))

### Blockchain

In [55]:
class Blockchain:
    def __init__(self, genesis_data, difficulty, blockReward=10):
        self.chain = []
        self.chain.append(Block(genesis_data))
        self.difficulty = difficulty
        self.size = 0
        self.block_reward = blockReward
        self.uncles = []

    def longestChain(self):
      max = self.chain[0].height
      for block in self.chain:
          if block.height > max:
                max = block.height
      maxes = [block for block in self.chain if block.height == max]
      r = random.choices(maxes, k=1)
      return r[0]
        
    def add(self, newBlock):
        self.chain.append(newBlock)
        newBlock.previous.children.append(newBlock)
        self.size +=1
        if self.inlongestChain(newBlock):
            newBlock.creator.reward += self.block_reward
        else:  
            self.uncles.append(newBlock)
        
    def print(self):
        for block in self.chain:
            block.print()
            print("________")

    def hasFork(self):
        for block1 in self.chain:
            for block2 in self.chain:
                if block1!=block2 and block1.height == block2.height:
                    return True
        return False

    def checkMiner(self, miner):
      last = self.longestChain()
      count = 0
      while last!=None:
          if last.creator == miner:
                count += 1
          last = last.previous
      return count
    
    def inlongestChain(self, block):
          last = self.longestChain()
          while last!= None:
                if last == block: #need more of a check for if fork
                        # print("last",last.height,"block", block.height)
                        return True
                last = last.previous
          return False

### Uncle blockchain

In [56]:
class UncleBlockchain(Blockchain):
    def __init__(self, genesis_data, difficulty, block_reward=10, uncle_reward=5):
        super().__init__(genesis_data, difficulty, blockReward=block_reward)
        self.uncle_reward = uncle_reward
        self.uncles = []
    
    def add(self, new_block):
        self.chain.append(new_block)
        new_block.previous.children.append(new_block)
        self.size +=1
        if self.inlongestChain(new_block):
            new_block.creator.reward += self.block_reward
            if self.uncles and new_block.height > self.uncles[0].height:
                self.referenceUncle()
        else:  
            self.uncles.append(new_block)

    def referenceUncle(self):
        uncleblock = self.uncles[-1]
        uncleblock.creator.reward += self.uncle_reward
        self.uncles.pop()

## Miners

### Normal miner

In [57]:
class Miner:
    def __init__(self, miningPower, name, blockchain=None):
        self.miningPower = miningPower
        # self.nonce = random.randint(0,100000) #original one
        self.nonce = 0
        self.name = name
        self.blockchain = blockchain
        self.reward = 0
        self.color = "#7EDBF6"
        if self.blockchain != None:
            self.lastBlock = blockchain.longestChain()
    
    def UpdateLast(self):
        latest = self.blockchain.longestChain()
        if latest.height > self.lastBlock.height:
              self.lastBlock = latest

    def PoWSolver(self):
        for i in range (0 , self.miningPower):
            newBlock = Block(str(self.blockchain.size), self, self.lastBlock, self.nonce)
            h = newBlock.hash_block()
            count = numberOfInitZeros(h)
            if count >= self.blockchain.difficulty:
                self.blockchain.add(newBlock)
                self.lastBlock = newBlock
            self.nonce += 1


### Selfish miner

In [58]:

class SelfishMiner(Miner):
    def __init__(self, miningPower, name, blockchain=None):
        super().__init__(miningPower, name, blockchain)
        self.privateBlocks = []
        self.publishNext = False
        self.color = "#F59AEE"

    def UpdateLast(self):
        #add this function
        latest = self.blockchain.longestChain()
        publicheight = latest.height
        if publicheight > self.lastBlock.height:
            self.privateBlocks = []
            self.lastBlock = latest
            self.publishNext = False
        if publicheight == self.lastBlock.height-1 and len(self.privateBlocks)> 1:
            for block in self.privateBlocks:
                self.blockchain.add(block)
                self.privateBlocks = []
        if publicheight == self.lastBlock.height:
            for block in self.privateBlocks:
                self.blockchain.add(block)
                self.privateBlocks = []
                self.publishNext = True

    def PoWSolver(self):
        #add this function
        for i in range (0 , self.miningPower):
            newBlock = Block(str(self.blockchain.size), self, self.lastBlock, self.nonce)
            h = newBlock.hash_block()
            count = numberOfInitZeros(h)
            if count >= self.blockchain.difficulty:
                if self.publishNext:
                    self.blockchain.add(newBlock)
                    self.publishNext = False
                else:
                    self.privateBlocks.append(newBlock)
                self.lastBlock = newBlock
            self.nonce += 1

## Main program

In [59]:
#Main
def run(
        reward=10, uncle_reward=0,
        difficulty=10,
        num_miners=10,
        num_selfish_miners=0,
        mining_power=10,
        mining_power_selfish=15,
        genesis_data="0",
    ):
    bc = None
    if uncle_reward > 0:
        bc = UncleBlockchain(genesis_data , difficulty, reward, uncle_reward)
    else:
        bc = Blockchain(genesis_data , difficulty, reward)
    miners = []
    selfish = []
    for i in range(num_miners):
        miners.append(Miner(mining_power ,"m"+str(i), bc))
    for i in range(num_selfish_miners):
        selfish.append(SelfishMiner(mining_power_selfish ,"s"+str(i), bc))
    while bc.size < 30:
        for s in selfish:
            s.PoWSolver()

        for m in miners:
            m.PoWSolver()

        for s in selfish:
            s.UpdateLast()

        for m in miners:
            m.UpdateLast()
    return bc, miners, selfish

def print_run(bc, miners, selfish):
    print("Has fork:", bc.hasFork())
    longest_chain = bc.longestChain()
    print("Longest chain height:",longest_chain.height)
    selfish_power = 0
    for s in selfish:
        selfish_power+=s.miningPower
        print(s.name, bc.checkMiner(s))
        print("Fraction {}".format(bc.checkMiner(s) /longest_chain.height ))
    total = selfish_power
    for m in miners:
        total += m.miningPower
    print("alpha {}".format(selfish_power / total))
    for m in miners:    
        print(m.name ,m.reward)
    for s in selfish:    
        print(s.name ,s.reward)
        
bc, m, s = run(num_selfish_miners=1)
print_run(bc, m, s)

Has fork: True
Longest chain height: 29
s0 3
Fraction 0.10344827586206896
alpha 0.13043478260869565
m0 30
m1 20
m2 0
m3 20
m4 50
m5 20
m6 20
m7 40
m8 50
m9 30
s0 30


In [60]:
show(bc)